## Neural Network
이번 장에서는 머신러닝의 한 방법인 신경망 Neural network을 tensorflow를 이용하여 설계하고 훈련하는 방법을 실습한다. 이를 이용하여 MNIST라 불리는, 0부터 9까지의 10개의 손으로 쓴 숫자를 구분하는 classification 예제를 구현한다.

* 본 실습의 예제는 tensorflow.org와 codelab.developers.google.com에서 나온 예제들을 수정하여 만들어졌습니다.

<p style="text-align:center;">MNIST dataset: Handwritten Digits</p>

![title](mnistExamples.png)

출처: simonwinder.com/

MNIST는 Yann LeCun 등의 학자들이 사람이 손으로 쓴 0에서 9까지의 숫자들을 모아서, 표준화, 사이즈 조정 등을 거쳐 만든 dataset이다. 약 6만개의 training set과 1만개의 test set이 있다. 비교적 단순하고 크기가 작다는 점 때문에 이미지 인식 분야에서 입문용으로 많이 쓰는 dataset이다. 위의 그림은 이 데이터의 일부를 모아서 다시 이미지로 표현한 것이다. 

MNIST 데이터를 직접 홈페이지에서 다운로드 받아도 되지만, tensorflow에서 직접 다운로드 받고 압축을 풀 수도 있다. 다음의 방식으로 데이터를 불러오자.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


데이터를 불러오는 데는 다소 시간이 걸린다. 정상적으로 처리되었다면 위의 예제처럼 다운로드 받은 후에 압축을 풀게 된다. 불러들인 데이터는 mnist라는 이름으로 저장되며, 이 중에 training data는 mnist.train, test data는 mnist.test와 같이 접근하면 된다. 각각은 다시 .images와 .labels로 이미지와 레이블에 접근할 수 있다.

In [3]:
print(mnist.test.images)
print('shape:', mnist.test.images.shape)
print('mean:' , mnist.test.images.mean())
print('min:' , mnist.test.images.min())
print('max:' , mnist.test.images.max())

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
shape: (10000, 784)
mean: 0.132515
min: 0.0
max: 1.0


위와 같이 image는 0부터 1까지 숫자들의 배열로 이루어져 있다. 이는 원래 그림의 pixel 값에서 온 것인데, MNIST 원래 image의 크기는 28x28, 즉 가로 28 픽셀, 세로 28픽셀인데 여기서는 이 image를 일렬로 쭉 늘려서 28*28=784의 크기로 다시 정렬한 것이다. 이렇게 정렬된 이미지가 10000개 존재하므로 사이즈가 (10000, 784)가 나왔다. 이렇게 일렬로 늘린 이유는 neural network의 input layer에 대입하기 위함인데, 이 내용은 아래에서 설명한다.

In [4]:
print(mnist.test.labels)
print('shape:', mnist.test.labels.shape)
print('\n', mnist.test.labels[0])

[[ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
shape: (10000, 10)

 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


이번에는 label을 살펴 보는데, label 역시 10000개의 test data 각각에 다 붙어있다. 그런데 한 label의 크기가 10인데, 예를 들어 첫번째 label을 출력해 보면 (위의 마지막 줄), 0과 1로 이루어진 10개의 원소를 가진 배열(벡터)이다. 우리가 구분할 label이 0부터 9까지의 숫자이기는 하지만, 그 성격을 생각해 보면 숫자로서가 아니라 10개 중의 하나에 해당하는 class로 보는 것이 더 합당하다. 따라서 우리가 할 것은 0에서 9까지의 수를 맞추는 regression이 아닌, 10개의 class 중 하나를 선택하는 classification 문제가 되며, 이를 위해서 mnist.test.labels에서는 숫자를 one-hot encoding이라는 방법을 써서 위와 같이 벡터로 표현했다. one-hot encoding에서는 표현하고자 하는 수에 해당하는 원소만 1로 하고, 나머지는 전부 0으로 만든다. 예를 들어 위의 벡터는 8번째 원소만 1이고 나머지는 다 0인데, 0부터 시작해서 8번째의 수, 즉 7을 나타낸다. 이 역시 neural network output layer에 맞추기 위한 방법이다.

정리하면, input image는 28x28을 길게 늘여뜨린 784개의 원소를 가진 벡터이고 각각의 원소는 0에서 1까지의 연속적인 값을 가진다. output label은 one-hot encoding을 사용해서, 10개의 원소를 가진 벡터이고 해당하는 class의 값만 1을 가진다. 이를 그림으로 표현하면 다음과 같다.

![](mnist_layers.png)
출처: codelabs.developers.google.com

위의 그림에서 input layer에 있는 784개의 원소들을 대입할 수 있는 "cell"을 neural network에서는 "neuron"이라 부른다. 마찬가지로 아래의 output layer에 있는 10개의 원소들을 대입할 수 있는 neuron들이 있다. 모든 input layer의 neuron들과 모든 output layer에 있는 neuron들은 서로 연결되어 있다(그림의 선들). 모든 neuron들이 연결되어 있기 때문에 이러한 구조를 fully-connected neural network라 부른다. 각각의 연결에는 weight가 부여된다. 

이 구조는 중간에 hidden later가 없이, input layer와 output layer가 직접 연결된 가장 단순한 simple neural network 구조를 보여준다. 이 때에 neural network를 "학습시킨다"라는 의미는 원하는 output에 가장 가까운 값을 주기 위한 weight들을 찾는다는 것을 의미한다. 이 예제에서는 784x10 = 7840개의 weight가 존재한다. 이는 784개의 행과 10개의 열을 가진 weight matrix로 생각할 수 있다.

이제 tensorflow를 사용하여 실제로 neural network를 학습시켜 보자. 출발은 앞서와 마찬가지로 데이터를 담을 placeholder를 지정하면서 시작한다.

In [5]:
import tensorflow as tf

X = tf.placeholder(tf.float32, [None, 784], name='image')
Y = tf.placeholder(tf.float32, [None, 10], name='label')

첫번째 인자로 데이터 타입을 넣고, 두번째 인자로 shape (dimension)을 정의한다. 이때 첫번째 원소에 "None"이 들어 있는 것에 주목하자. 우리에겐 6만개의 training data가 있지만, 이들 모두를 한번에 대입하지는 않을 것이다. 대신에 여러 개의 데이터 샘플 "덩어리"를 집어 넣는 방법을 반복할 것이다. 이를 batch라고 부른다. 이 때 batch의 크기, 즉 한꺼번에 대입할 데이터의 갯수는 바뀔 수 있기 때문에 "None"이라고 지정해서 나중에 어떤 크기의 데이터도 대입 가능하게 해 준다. 

In [6]:
X.get_shape()

TensorShape([Dimension(None), Dimension(784)])

![](206327168bc85294.png)
출처: codelabs.developers.google.com

우리가 훈련시킬 neural network 모델을 식으로 표현하면 위와 같다. 그림에서 이미지의 차원이 [100, 784]인 것은 예제에서 batch의 크기를 100으로 잡았기 때문이다. 즉 한꺼번에 100개의 이미지를 입력한다는 뜻이 된다. 그러나 이 값은 자유롭게 바꿀 수 있기에 크게 중요한 것은 아니다. 중요한 숫자는 이미지 하나의 크기인 784와 이에 곱해지는 weight matrix의 사이즈 [784, 10]이다. 매트릭스 곱, 더 일반적으로 텐서 곱에서 중요한 것은, 곱해지는 텐서들의 차원이 맞아야 한다는 점이다. 곱하기 XW에서 중요한 것은 X의 열의 크기인 784와 W의 행의 크기인 784가 일치해야 한다. 그리고 W의 열의 크기 10은 결과값이 one-hot encoding을 한 크기 10의 벡터 Y이기 때문이다. 마찬가지로, XW의 결과에 더해지는 bias인 b의 크기도 10이다. 이때, b는 벡터로 취급되며 Y가 [100, 10]의 크기를 가지는 것과는 다르게 그냥 [10]의 크기를 가짐을 주목하자. tensorflow에서는 bias의 경우 batch size를 포함하지 않는다. 실제로는 각 batch마다 동일한 bias가 더해지기 때문이다. weight와 bias의 크기를 확인했으면, 이제 이들을 tf.Variable로 정의하자.

In [7]:
W = tf.Variable(tf.zeros([784, 10]), name='weight')
b = tf.Variable(tf.zeros([10]), name='bias')

위의 model에서 softmax는 output neuron의 activation function이다. activation function은 간단히 얘기해서 y=f(x)와 같은 함수의 형태에서 input x를 output y로 변환하는 함수이다. 여기에서는 softmax라는 activation function을 사용한다. softmax의 결과는 항상 0에서 1까지의 실수이며, 가능한 경우를 다 더했을 때 1이 된다. 즉, 이 경우는 10개의 output cell들의 결과들을 다 더하면 1이 된다. 이는 softmax의 결과를 특정 class에 속할 (즉 특정 숫자를 나타내는) 확률로 해석 가능하게 한다. 우리는 나중에 model의 예측을 평가하기 위해 softmax의 결과들 중 최대치를 가지는 cell을 잡아서, 그 class(숫자)를 예측한다고 할 것이다. softmax는 tensorflow에서 tf.nn.softmax로 제공되며, X와 W의 곱은 tf.matmul을 사용하여 계산한다.

In [8]:
linear_output = tf.matmul(X, W) + b
Y_prediction = tf.nn.softmax(linear_output)

이제 model이 완성되었으므로 예측값과 실제 데이터의 y를 비교하여, 예측이 얼마나 정확한지를 평가하는 loss function을 지정할 차례이다. one-hot encoding과 같이 vector 형태의 결과를 비교할 때는 주로 cross-entropy라는 loss function이 자주 쓰인다. 아래 예제에서 Y_i'은 실제 데이터의 값이고 log(Y_i)에서 Y_i는 softmax의 결과물이다. 예제에서처럼 올바른 class에 높은 확률을 줄 수록 cross-entropy의 값은 작아지게 된다. 우리는 neural network을 훈련시켜 cross-entropy를 최소화하기 위해 노력할 것이다.

![](1d8fc59e6a674f1c.png)
출처: codelabs.developers.google.com

In [11]:
cross_entropy = -tf.reduce_sum(Y * tf.log(Y_prediction), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)

위 코드에서 reduction_indices=[1]의 의미는 Y와 Y_prediction의 element 곱 이후의 합을 열을 따라 한다는 뜻이다. Y의 크기가 [100, 10]임을 상기하자. 100은 batch의 크기이고 0의 차원이며, 10은 one-hot encoded vector의 크기고 1의 차원이다. 실제로 우리가 사용하는 loss는 이렇게 구한 cross-entropy의 평균이다. 

이제 loss를 최소화 하기 위한 optimizer를 정의한다. 앞서와 마찬가지로 gradient descent optimzer를 사용하며 학습 속도는 0.5로 잡자. 이어서 session을 생성하고 Variable initialization을 시행한다.

In [12]:
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

In [13]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

훈련을 위해 1000번의 loop를 설정하고, 매번 100개의 데이터를 읽어서 batch를 만들어서 train 시키자.

In [16]:
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train, feed_dict={X: batch_xs, Y: batch_ys})

훈련이 끝났으면 정확도를 계산해 보자. 정확도는 매 샘플마다 해당 class를 정확하게 맞추었을 경우 1, 그렇지 않을 경우 0으로 계산한 다음, 전체에 대해 평균을 내어 구한다. 이를 위해 실제 Y와 예측 Y 모두 10개의 원소들 중 최대값에 해당하는 인덱스를 돌려 주는 tf.argmax를 사용하여 다음 코드의 첫번째 줄처럼 tf.equal을 사용하여 둘의 값이 같은 지 다른 지를 비교한다. tf.eqaul은 True or False를 return하므로 tf.cast를 이용하여 1 or 0으로 바꾸어 준 다음에, tf.reduce_mean을 통해 평균을 구한다.

In [29]:
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_prediction, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('test accuracy: ')
print(sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

test accuracy: 
0.9151
